In [1]:
include("../../src/struct_data.jl")
methods(force_func)

# 6 methods for generic function "force_func" from Main:
 [1] force_func(p::Cubic, value)
     @ none:1
 [2] force_func(p::Cubic, i, r)
     @ ~/Documents/MaestriaUNAL/CellAggregate.jl/src/forces/forces_func.jl:31
 [3] force_func(p::LennardJones, value)
     @ none:1
 [4] force_func(p::LennardJones, i, r)
     @ ~/Documents/MaestriaUNAL/CellAggregate.jl/src/forces/forces_func.jl:44
 [5] force_func(p::Oriola, value)
     @ none:1
 [6] force_func(p::Oriola, i, r)
     @ ~/Documents/MaestriaUNAL/CellAggregate.jl/src/forces/forces_func.jl:57

# Force

In [2]:
dump(Cubic)

UnionAll
  var: TypeVar
    name: Symbol T
    lb: Union{}
    ub: Any
  body: Cubic{T} <: ForceType
    μ₁::T
    rₘᵢₙ::T
    rₘₐₓ::T


# Model

In [3]:
Model = ModelSet(
    TimeModel(
        tₛᵢₘ  = 10000.0,
        dt    = 0.5,
        nₖₙₙ  = 50,
        nₛₐᵥₑ = 100
    ),
    InputModel(
        outer_ratio = 0.8,
        path_input  = "../../data/init/Sphere"
    ),
    OutputModel(
        name_output = "Test_1",
        path_output = ".results/cont_par(A)/force_par(B)/",
        d_saved = 0.5
    ) 
)
dump(Model)

ModelSet
  Time: TimeModel
    tₛᵢₘ: Float64 10000.0
    dt: Float64 0.5
    nₖₙₙ: Int64 50
    nₛₐᵥₑ: Int64 100
  Input: InputModel
    outer_ratio: Float64 0.8
    path_input: String 

"../../data/init/Sphere"
  Output: OutputModel
    name_output: String "Test_1"
    path_output: String ".results/cont_par(A)/force_par(B)/"
    d_saved: Float64 0.5


# Data

In [4]:
X = Float64.(readdlm("../../data/init/Sphere/15.0.xyz")[3:end,2:end]) |> cu

2504×3 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 -1.5  -4.62  -13.88
  0.5  -4.62  -13.88
  2.5  -4.62  -13.88
 -4.5  -2.89  -13.88
 -2.5  -2.89  -13.88
 -0.5  -2.89  -13.88
  1.5  -2.89  -13.88
  3.5  -2.89  -13.88
 -5.5  -1.15  -13.88
 -3.5  -1.15  -13.88
  ⋮           
  4.5   1.15   13.88
 -4.5   2.89   13.88
 -2.5   2.89   13.88
 -0.5   2.89   13.88
  1.5   2.89   13.88
  3.5   2.89   13.88
 -1.5   4.62   13.88
  0.5   4.62   13.88
  2.5   4.62   13.88

# Aggregate

## One

In [5]:
agg = Aggregate(
    [AggType("HEK_1", InteractionPar(Cubic(1.0,2.0,3.0), ContractilePar(0.01,pi/4,0.08,1.0)),X)],
    [AggLocation("HEK_1",[0 0 0])],
    Model
)

# dump(agg)
show_aggregates(agg)
clean_agg(agg)

1-element Vector{AggType}:
 AggType("HEK_1", InteractionPar(Cubic{Float64}(1.0, 2.0, 3.0), ContractilePar{Float64}(0.01, 0.7853981633974483, 0.08, 1.0)), 15.27f0, Float32[-1.5 -4.62 -13.88; 0.5 -4.62 -13.88; … ; 0.5 4.62 13.88; 2.5 4.62 13.88], CuArray{Float32, 2, CUDA.Mem.DeviceBuffer})

========================= Type =======================


1×2504 CuArray{Int64, 2, CUDA.Mem.DeviceBuffer}:
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  1  1  1  1  1  1  1  1  1  1  1  1

======================   Index =======================
Index of List of Aggregates


1×2504 CuArray{Int64, 2, CUDA.Mem.DeviceBuffer}:
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  1  1  1  1  1  1  1  1  1  1  1  1

...

ContractilePar{CuArray{Float32, 1, CUDA.Mem.DeviceBuffer}}(Float32[0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01  …  0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01], Float32[0.7853982, 0.7853982, 0.7853982, 0.7853982, 0.7853982, 0.7853982, 0.7853982, 0.7853982, 0.7853982, 0.7853982  …  0.7853982, 0.7853982, 0.7853982, 0.7853982, 0.7853982, 0.7853982, 0.7853982, 0.7853982, 0.7853982, 0.7853982], Float32[0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08  …  0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08], Float32[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0])

Contractile


1-element CuArray{Float32, 1, CUDA.Mem.DeviceBuffer}:
 15.27

Radius


------------------ Neighbors Size -------------------
idx_red  = (1000, 2504)
idx_sum  = (1, 2504)
idx_cont = (50, 2504)
------------------- Forces Size ---------------------
dX       = (2504, 3)
F        = (2504, 3)


## Simple Agg

In [6]:
@start_agg Simple = Aggregate(
    [AggType("HEK_1", InteractionPar(Cubic(1.0,2.0,3.0), ContractilePar(0.01,pi/4,0.08,1.0)),X)],
    [
        AggLocation("HEK_1",[100 0   0]),
        AggLocation("HEK_1",[0   100 0])
    ],
    Model
)
show_aggregates(Simple)
clean_agg(Simple)

1-element Vector{AggType}:
 AggType("HEK_1", InteractionPar(Cubic{Float64}(1.0, 2.0, 3.0), ContractilePar{Float64}(0.01, 0.7853981633974483, 0.08, 1.0)), 15.27f0, Float32[-1.5 -4.62 -13.88; 0.5 -4.62 -13.88; … ; 0.5 4.62 13.88; 2.5 4.62 13.88], CuArray{Float32, 2, CUDA.Mem.DeviceBuffer})

1×5008 CuArray{Int64, 2, CUDA.Mem.DeviceBuffer}:
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  1  1  1  1  1  1  1  1  1  1  1  1

1×5008 CuArray{Int64, 2, CUDA.Mem.DeviceBuffer}:
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  2  2  2  2  2  2  2  2  2  2  2  2

1×5008 Matrix{String}:
 "HEK_1"  "HEK_1"  "HEK_1"  "HEK_1"  …  "HEK_1"  "HEK_1"  "HEK_1"  "HEK_1"

5008×3 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
  98.5   -4.62  -13.88
 100.5   -4.62  -13.88
 102.5   -4.62  -13.88
  95.5   -2.89  -13.88
  97.5   -2.89  -13.88
  99.5   -2.89  -13.88
 101.5   -2.89  -13.88
 103.5   -2.89  -13.88
  94.5   -1.15  -13.88
  96.5   -1.15  -13.88
   ⋮            
   4.5  101.15   13.88
  -4.5  102.89   13.88
  -2.5  102.89   13.88
  -0.5  102.89   13.88
   1.5  102.89   13.88
   3.5  102.89   13.88
  -1.5  104.62   13.88
   0.5  104.62   13.88
   2.5  104.62   13.88

...

1×5008 Matrix{Int64}:
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  1  1  1  1  1  1  1  1  1  1  1  1

Cubic{CuArray{Float32, 1, CUDA.Mem.DeviceBuffer}}(Float32[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], Float32[2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0  …  2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0], Float32[3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0  …  3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0])

ContractilePar{CuArray{Float32, 1, CUDA.Mem.DeviceBuffer}}(Float32[0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01  …  0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01], Float32[0.7853982, 0.7853982, 0.7853982, 0.7853982, 0.7853982, 0.7853982, 0.7853982, 0.7853982, 0.7853982, 0.7853982  …  0.7853982, 0.7853982, 0.7853982, 0.7853982, 0.7853982, 0.7853982, 0.7853982, 0.7853982, 0.7853982, 0.7853982], Float32[0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08  …  0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08], Float32[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0])

1-element CuArray{Float32, 1, CUDA.Mem.DeviceBuffer}:
 15.27

========================= Type =======================
======================   Index =======================
Index of List of Aggregates
Index of Number of Aggregates
Index of Name of Aggregates
====================== Position =====================
======================== Geometry ===================
Radius_agg
Outline
Outer/Total = 1.0
====================== Simulation ===================
---------------------- Parameter --------------------
Force
Contractile
Radius
------------------ Neighbors Size -------------------
idx_red  = (1000, 5008)
idx_sum  = (1, 5008)
idx_cont = (50, 5008)
------------------- Forces Size ---------------------
dX       = (5008, 3)
F        = (5008, 3)


## Complex

In [7]:
println("================================== Aggregate ====================================")
X = Float64.(readdlm("../../data/init/Sphere/15.0.xyz")[3:end,2:end]) |> cu
Y = Float64.(readdlm("../../data/init/Sphere/10.0.xyz")[3:end,2:end]) |> cu

@start_agg Complex = Aggregate(
    [
        AggType("A", InteractionPar(Cubic(1.0,2.0,3.0), ContractilePar(0.01,pi/4,0.08,1.0)),X),
        AggType("B", InteractionPar(Cubic(2.0,2.0,3.0), ContractilePar(0.02,pi/4,0.08,1.0)),Y),
        AggType("C", InteractionPar(Cubic(3.0,2.0,3.0), ContractilePar(0.03,pi/4,0.08,1.0)),X),
        AggType("D", InteractionPar(Cubic(4.0,2.0,3.0), ContractilePar(0.04,pi/4,0.08,1.0)),Y),
    ],
    [
        AggLocation("A",[100 0   0  ]),
        AggLocation("C",[0   100 0  ]),
        AggLocation("D",[0   100 100]),
        AggLocation("B",[0   100 0  ])
    ],
    Model
)
show_aggregates(Complex)
clean_agg(Complex)

================================== Aggregate ====================================


4-element Vector{AggType}:
 AggType("A", InteractionPar(Cubic{Float64}(1.0, 2.0, 3.0), ContractilePar{Float64}(0.01, 0.7853981633974483, 0.08, 1.0)), 15.27f0, Float32[-1.5 -4.62 -13.88; 0.5 -4.62 -13.88; … ; 0.5 4.62 13.88; 2.5 4.62 13.88], CuArray{Float32, 2, CUDA.Mem.DeviceBuffer})
 AggType("B", InteractionPar(Cubic{Float64}(2.0, 2.0, 3.0), ContractilePar{Float64}(0.02, 0.7853981633974483, 0.08, 1.0)), 10.43f0, Float32[-1.5 -2.89 -8.98; 0.5 -2.89 -8.98; … ; 0.5 2.89 8.98; 2.5 2.89 8.98], CuArray{Float32, 2, CUDA.Mem.DeviceBuffer})
 AggType("C", InteractionPar(Cubic{Float64}(3.0, 2.0, 3.0), ContractilePar{Float64}(0.03, 0.7853981633974483, 0.08, 1.0)), 15.27f0, Float32[-1.5 -4.62 -13.88; 0.5 -4.62 -13.88; … ; 0.5 4.62 13.88; 2.5 4.62 13.88], CuArray{Float32, 2, CUDA.Mem.DeviceBuffer})
 AggType("D", InteractionPar(Cubic{Float64}(4.0, 2.0, 3.0), ContractilePar{Float64}(0.04, 0.7853981633974483, 0.08, 1.0)), 10.43f0, Float32[-1.5 -2.89 -8.98; 0.5 -2.89 -8.98; … ; 0.5 2.89 8.98; 2.5 2.89 

1×6500 CuArray{Int64, 2, CUDA.Mem.DeviceBuffer}:
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  2  2  2  2  2  2  2  2  2  2  2  2

1×6500 CuArray{Int64, 2, CUDA.Mem.DeviceBuffer}:
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  4  4  4  4  4  4  4  4  4  4  4  4

1×6500 Matrix{String}:
 "A"  "A"  "A"  "A"  "A"  "A"  "A"  "A"  …  "B"  "B"  "B"  "B"  "B"  "B"  "B"

...

1×6500 Matrix{Int64}:
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  1  1  1  1  1  1  1  1  1  1  1  1

Cubic{CuArray{Float32, 1, CUDA.Mem.DeviceBuffer}}(Float32[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0], Float32[2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0  …  2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0], Float32[3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0  …  3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0])

ContractilePar{CuArray{Float32, 1, CUDA.Mem.DeviceBuffer}}(Float32[0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01  …  0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02], Float32[0.7853982, 0.7853982, 0.7853982, 0.7853982, 0.7853982, 0.7853982, 0.7853982, 0.7853982, 0.7853982, 0.7853982  …  0.7853982, 0.7853982, 0.7853982, 0.7853982, 0.7853982, 0.7853982, 0.7853982, 0.7853982, 0.7853982, 0.7853982], Float32[0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08  …  0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08], Float32[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0])

4-element CuArray{Float32, 1, CUDA.Mem.DeviceBuffer}:
 15.27
 10.43
 15.27
 10.43

========================= Type =======================
======================   Index =======================
Index of List of Aggregates
Index of Number of Aggregates
Index of Name of Aggregates
====================== Position =====================
======================== Geometry ===================
Radius_agg
Outline
Outer/Total = 1.0
====================== Simulation ===================
---------------------- Parameter --------------------
Force
Contractile
Radius
------------------ Neighbors Size -------------------
idx_red  = (1000, 6500)
idx_sum  = (1, 6500)
idx_cont = (50, 6500)
------------------- Forces Size ---------------------
dX       = (6500, 3)
F        = (6500, 3)
